In [1]:
from tomoSegmentPipeline.showcaseResults import (predict_fullTomogram, load_model, load_tomoData, Tversky_index,
                                        fullTomogram_modelComparison, make_comparison_plot, write_comparison_gif, save_classPred)

from tomoSegmentPipeline.losses import Tversky_loss
from tomoSegmentPipeline.utils.common import read_array
import tomoSegmentPipeline.dataloader as dl
from tomoSegmentPipeline.utils import setup
from tomoSegmentPipeline.dataloader import to_categorical, transpose_to_channels_first
from tomoSegmentPipeline.trainWrapper import make_trainer

PARENT_PATH = setup.PARENT_PATH

import numpy as np
import matplotlib.pyplot as plt
import random
import mrcfile
import pandas as pd
import torch
import os
from glob import glob
import random

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

train_tomos = ['tomo02', 'tomo03', 'tomo17']
concat_train_ids = sorted([s.replace('tomo', '') for s in train_tomos])
concat_train_ids = '-'.join(concat_train_ids)

val_tomos = ['tomo32', 'tomo10']
concat_val_ids = sorted([s.replace('tomo', '') for s in val_tomos])
concat_val_ids = '-'.join(concat_val_ids)

test_tomos = ['tomo38', 'tomo04']
concat_test_ids = sorted([s.replace('tomo', '') for s in test_tomos])
concat_test_ids = '-'.join(concat_test_ids)


paths_trainData, paths_trainTarget = setup.get_paths(train_tomos, 'cryoCARE')
paths_valData, paths_valTarget = setup.get_paths(val_tomos, 'cryoCARE')
paths_testData, paths_testTarget = setup.get_paths(test_tomos, 'cryoCARE')

In [ ]:
from tomoSegmentPipeline import dataloader as dl
from tomoSegmentPipeline.utils import setup
from tomoSegmentPipeline.dataloader import to_categorical, transpose_to_channels_first, tomoSegment_dummyDataset, tomoSegment_dataset
from tomoSegmentPipeline.training import Train
import os

PARENT_PATH = setup.PARENT_PATH

import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


val_tomos = ['tomo32', 'tomo10']
concat_val_ids = sorted([s.replace('tomo', '') for s in val_tomos])
concat_val_ids = '-'.join(concat_val_ids)

test_tomos = ['tomo38', 'tomo04']
concat_test_ids = sorted([s.replace('tomo', '') for s in test_tomos])
concat_test_ids = '-'.join(concat_test_ids)

train_tomos = ['tomo02']

concat_train_ids = sorted([s.replace('tomo', '') for s in train_tomos])
concat_train_ids = '-'.join(concat_train_ids)

chkpnt = None

tb_logdir = os.path.join(PARENT_PATH, 'models_scratchpad2/earlyStop_tryout/logs/LowBaselineModel/train%s' %concat_train_ids)
model_name = '3.09_lowBaseline'
epochs = 100


Ncl = 2
dim_in = 56
lr = 1e-2
weight_decay = 0
Lrnd = 0
augment_data = False
batch_size = 64
nPatches_training = 1
pretrained_model = None

trainer = Train(Ncl=Ncl, dim_in=dim_in, lr=lr, weight_decay=weight_decay, Lrnd=Lrnd, tensorboard_logdir=tb_logdir,
                model_name=model_name, augment_data=augment_data, batch_size=batch_size, epochs=epochs,
                pretrained_model=pretrained_model)

early_stop_callback = EarlyStopping(
            monitor='hp/val_loss_epoch',
            min_delta=0.5,
            patience=8,
            verbose=True,
            mode='min',
            # check_on_train_epoch_end=False
            )

trainer.launch(train_tomos, val_tomos, input_type='rawCET', num_gpus=1, accelerator=None,
               num_workers=1, resume_from_checkpoint=chkpnt,  nPatches_training=nPatches_training,
               train_callbacks=[early_stop_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Set SLURM handle signals.

  | Name    | Type         | Params
-----------------------------------------
0 | loss_fn | Tversky_loss | 0     
1 | layer1  | Sequential   | 28.6 K
2 | layer2  | Sequential   | 103 K 
3 | layer3  | Sequential   | 558 K 
4 | layer4  | Sequential   | 288 K 
5 | layer5  | Sequential   | 96.9 K
-----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.304     Total estimated model params size (MB)


"Ncl":           2
"loss_fn":       Tversky_loss()
"lr":            0.01
"pretrain_type": None
"weight_decay":  0


Metric hp/val_loss_epoch improved. New best score: 1.064


In [16]:
trainer.model.lr_schedulers()

In [13]:
from tomoSegmentPipeline.showcaseResults import (predict_fullTomogram, load_model, load_tomoData, Tversky_index,
                                        fullTomogram_modelComparison, make_comparison_plot, write_comparison_gif, save_classPred)

model_file = '/home/haicu/jeronimo.carvajal/Thesis/models_scratchpad2/earlyStop_tryout/logs/LowBaselineModel/train02/version_29/3.09_lowBaseline_ep100_in56_lr0.010000_29.model'
model = load_model(model_file, 2)